In [23]:
from path import Path
import pandas as pd

from main import Worker

In [24]:
a = Path(r'.').glob(r'*.xlsx')
assert len(a) == 1

df_ = pd.read_excel(a[0], engine='openpyxl')

worker = Worker(df_)
df = worker.df

In [31]:
def get_description(row: pd.Series) -> str:
    return "-=] BESCHREIBUNG [=-"

In [32]:
from datetime import datetime
from dateutil import parser

row = df.iloc[0]

cur_date = row['Termin 1']
start, end = row['Uhrzeit'].split(' - ')

start_datetime = datetime.combine(cur_date.date(), parser.parse(start).time())
end_datetime = datetime.combine(cur_date.date(), parser.parse(end).time())

print(f'start      : {start_datetime}')
print(f'end        : {end_datetime}')
print(f'subject    : {row["Kurs"].replace(" -", "-").replace("- ", "-")}')
print(f'description: {get_description(row)}')

print()

start      : 2022-01-03 19:00:00
end        : 2022-01-03 21:00:00
subject    : Aufbau-Kurs für Einsteiger
description: -=] BESCHREIBUNG [=-

